In [1]:
# Import current yahoo finance API, data manipulation, dash modules

import yfinance as yf
from datetime import date
from dateutil.relativedelta import relativedelta
import dash
import dash_core_components as dcc
import dash_html_components as html
import numpy as np
import pandas as pd
import math
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Output, Input

# relative_days determines date range for yfinance query

relative_days = 1000
x_days_ago = date.today() - relativedelta(days = relative_days)

# labels_list is used when generating datasets. Symbols are currently hardcoded, though ideally would be pulled live from a database.

labels = {0: 'GOOG'
, 1: 'NKE'
, 2: 'MSFT'
, 3: 'KO'
, 4: 'FB'
, 5: 'GME'
, 6: 'CRM'
, 7: 'DIS'
, 8: 'OTEX'
, 9: 'F'
, 10: 'SNAP'
, 11: 'MCD'
, 12: 'VTI'
, 13: 'AAPL'
, 14: 'NFLX'
, 15: 'TSLA'}
labels_list = list(labels.values())

ticker_tuples = ' '.join(labels_list)

tickers = yf.Tickers(ticker_tuples)

tickers_ls = list(tickers.tickers)


In [2]:
# v1 of Stonks builds dataframes for both price trends and x-day comparisons. 
# the bulk of Stonks' labor is performed in the dataframe building script.
# I'd like to cache this, make it a regular job, or find a more stable API for this data. ATM, free is ok.

def build_dataframes(ls, price_type):
    ticker_history = pd.DataFrame()
    ticker_prices = pd.DataFrame()
    pe_ratio = pd.DataFrame()
    for l, t in enumerate(ls):
        if l == 0:
            ticker_history = pd.DataFrame(tickers.tickers[tickers_ls[l]].history(start = x_days_ago)['Close']).rename(columns = {'Close': t})
            ticker_prices = pd.DataFrame({'ticker': t, 'two_hundred_day_average': [tickers.tickers[tickers_ls[l]].info['twoHundredDayAverage']]
                                            , 'previous_close': [tickers.tickers[tickers_ls[l]].info['previousClose']]
                                            , 'fifty_day_average': [tickers.tickers[tickers_ls[l]].info['fiftyDayAverage']]}
                                            , columns = ['ticker','two_hundred_day_average', 'previous_close', 'fifty_day_average'])
        if l > 0:
            ticker_history = ticker_history.merge(pd.DataFrame(tickers.tickers[tickers_ls[l]].history(start = x_days_ago)['Close']).rename(columns = {'Close': t})
                                    , how='inner'
                                    , left_index=True
                                    , right_index=True)
            ticker_prices = ticker_prices.append(pd.DataFrame({'ticker': t, 'two_hundred_day_average': [tickers.tickers[tickers_ls[l]].info['twoHundredDayAverage']]
                                            , 'fifty_day_average': [tickers.tickers[tickers_ls[l]].info['fiftyDayAverage']]
                                            , 'previous_close': [tickers.tickers[tickers_ls[l]].info['previousClose']]}
                                            , columns = ['ticker','two_hundred_day_average', 'fifty_day_average', 'previous_close']))
    if price_type == 'compare':
        return ticker_prices
    if price_type == 'history':
        return ticker_history

# final dataframes

trended_prices = build_dataframes(labels_list, 'history').reset_index()
new_prices = build_dataframes(labels_list, 'compare').reset_index()[['ticker','two_hundred_day_average', 'fifty_day_average', 'previous_close']]

compared_prices = new_prices.melt(id_vars=['ticker'], 
                     var_name=['Type'],
                     value_name='Value')

In [3]:
# Current reported fields: price history, recent price comparisons

external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
        "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server
app.title = "Stonks Monitoring"

app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.P(children="🤑", className="header-emoji"),
                html.H1(
                    children="Stonks", className="header-title"
                ),
                html.P(
                    children="Track the trended prices of stocks "
                    "and assess sell/buy activities.",
                    className="header-description",
                ),
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Symbol", className="menu-title"),
                        dcc.Dropdown(
                            id="symbol-filter",
                            options=[
                                {"label": symbol, "value": symbol}
                                for symbol in np.sort(tickers_ls)
                            ],
                            value="GOOG",
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Date Range",
                            className="menu-title"
                            ),
                        dcc.DatePickerRange(
                            id="date-range",
                            min_date_allowed='2001-01-01',
                            max_date_allowed=date.today(),
                            start_date=date.today() - relativedelta(days = 365),
                            end_date=date.today(),
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="trended-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
                html.Div(
                    children=dcc.Graph(
                        id="compared-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),
    ]
)

# To process filters and user input, we'll callback to the Yahoo API data source.
# The trend and price comparison charts should update simultaneously.
@app.callback(
    [Output("trended-chart", "figure"), Output("compared-chart", "figure")],
    [
        Input("symbol-filter", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ],
)
def update_charts(symbol, start_date, end_date):
    mask_trended = (
        (trended_prices.Date >= start_date)
        & (trended_prices.Date <= end_date)
    ) # the trended mask filters on date and symbol. 
    filtered_trended_prices = trended_prices.loc[mask_trended, ['Date', symbol]]
    trended_chart_figure = {
        "data": [
            {
                "x": filtered_trended_prices["Date"],
                "y": filtered_trended_prices[symbol],
                "type": "lines",
                "hovertemplate": "$%{y:.2f}<extra></extra>",
            },
        ],
        "layout": {
            "title": {
                "text": "Trended Price of the Stock",
                "x": 0.05,
                "xanchor": "left",
            },
            "xaxis": {"fixedrange": True},
            "yaxis": {"tickprefix": "$", "fixedrange": True},
            "colorway": ["#17B897"],
        },
    }
    mask_compared = (
        (compared_prices.ticker == symbol)
    ) # the comparison mask filters on symbol only. 

    filtered_compared_chart = compared_prices.loc[mask_compared, :]

    x=filtered_compared_chart['Type']
    y=filtered_compared_chart['Value']
    z=filtered_compared_chart['ticker']

    compared_chart_figure = go.Figure(
        data=[go.Bar(
        x=x,
        y=y,
        text=y,
        customdata=z, 
        textposition='outside',
        marker=dict(color="LightSeaGreen"),
        hovertemplate="<br>"
        .join(["Ticker: %{customdata}<extra></extra>",
               "Type: %{x}",
               "Value: %{y}"
              ]),
        texttemplate='%{y}'
        )])
    
    compared_chart_figure.update_yaxes(# Prices are in dollars
        tickprefix="$", showgrid=True
        )
    compared_chart_figure.update_layout(yaxis_title=None,
                  xaxis_title=None,
                  barmode='group',
                  template='plotly_white',)
    return trended_chart_figure, compared_chart_figure


#if __name__ == "__main__":
app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
%tb

SystemExit: 1